In [4]:
import numpy as np
import awkward0

In [9]:
cd ..

/home/rd804/ParticleNet


In [5]:
import logging
logging.basicConfig(level=logging.INFO, format='[%(asctime)s] %(levelname)s: %(message)s')

In [6]:
def stack_arrays(a, keys, axis=-1):
    flat_arr = np.stack([a[k].flatten() for k in keys], axis=axis)
    return awkward0.JaggedArray.fromcounts(a[keys[0]].counts, flat_arr)

In [7]:
def pad_array(a, maxlen, value=0., dtype='float32'):
    x = (np.ones((len(a), maxlen)) * value).astype(dtype)
    for idx, s in enumerate(a):
        if not len(s):
            continue
        trunc = s[:maxlen].astype(dtype)
        x[idx, :len(trunc)] = trunc
    return x

In [8]:
class Dataset(object):

    def __init__(self, filepath, feature_dict = {}, label='label', pad_len=100, data_format='channel_first'):
        self.filepath = filepath
        self.feature_dict = feature_dict
        if len(feature_dict)==0:
            feature_dict['points'] = ['part_etarel', 'part_phirel']
            feature_dict['features'] = ['part_pt_log', 'part_e_log', 'part_etarel', 'part_phirel']
            feature_dict['mask'] = ['part_pt_log']
        self.label = label
        self.pad_len = pad_len
        assert data_format in ('channel_first', 'channel_last')
        self.stack_axis = 1 if data_format=='channel_first' else -1
        self._values = {}
        self._label = None
        self._load()

    def _load(self):
        logging.info('Start loading file %s' % self.filepath)
        counts = None
        with awkward0.load(self.filepath) as a:
            self._label = a[self.label]
            for k in self.feature_dict:
                cols = self.feature_dict[k]
                if not isinstance(cols, (list, tuple)):
                    cols = [cols]
                arrs = []
                for col in cols:
                    if counts is None:
                        counts = a[col].counts
                    else:
                        assert np.array_equal(counts, a[col].counts)
                    arrs.append(pad_array(a[col], self.pad_len))
                self._values[k] = np.stack(arrs, axis=self.stack_axis)
        logging.info('Finished loading file %s' % self.filepath)


    def __len__(self):
        return len(self._label)

    def __getitem__(self, key):
        if key==self.label:
            return self._label
        else:
            return self._values[key]
    
    @property
    def X(self):
        return self._values
    
    @property
    def y(self):
        return self._label

    def shuffle(self, seed=None):
        if seed is not None:
            np.random.seed(seed)
        shuffle_indices = np.arange(self.__len__())
        np.random.shuffle(shuffle_indices)
        for k in self._values:
            self._values[k] = self._values[k][shuffle_indices]
        self._label = self._label[shuffle_indices]

In [10]:
train_dataset = Dataset('data/converted/train_file_0.awkd', data_format='channel_last')
val_dataset = Dataset('data/converted/val_file_0.awkd', data_format='channel_last')

[2023-04-07 06:01:38,070] INFO: Start loading file data/converted/train_file_0.awkd
[2023-04-07 06:02:37,025] INFO: Finished loading file data/converted/train_file_0.awkd
[2023-04-07 06:02:37,035] INFO: Start loading file data/converted/val_file_0.awkd
[2023-04-07 06:02:58,542] INFO: Finished loading file data/converted/val_file_0.awkd


In [11]:
import tensorflow as tf
from tensorflow import keras
from tf_keras_model import get_particle_net, get_particle_net_lite

In [12]:
model_type = 'particle_net_lite' # choose between 'particle_net' and 'particle_net_lite'
num_classes = train_dataset.y.shape[1]
input_shapes = {k:train_dataset[k].shape[1:] for k in train_dataset.X}
if 'lite' in model_type:
    model = get_particle_net_lite(num_classes, input_shapes)
else:
    model = get_particle_net(num_classes, input_shapes)

2023-04-07 06:05:51.745460: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15357 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:02:00.0, compute capability: 6.0
2023-04-07 06:05:51.755806: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13877 MB memory:  -> device: 1, name: Tesla P100-PCIE-16GB, pci bus id: 0000:81:00.0, compute capability: 6.0
2023-04-07 06:05:51.758944: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 12541 MB memory:  -> device: 2, name: Tesla P100-PCIE-16GB, pci bus id: 0000:82:00.0, compute capability: 6.0


In [14]:
# Training parameters
batch_size = 1024 if 'lite' in model_type else 384
epochs = 2

In [15]:
def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 10:
        lr *= 0.1
    elif epoch > 20:
        lr *= 0.01
    logging.info('Learning rate: %f'%lr)
    return lr

In [16]:
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adam(learning_rate=lr_schedule(0)),
              metrics=['accuracy'])
model.summary()

[2023-04-07 06:06:12,538] INFO: Learning rate: 0.001000


Model: "ParticleNet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 mask (InputLayer)              [(None, 100, 1)]     0           []                               
                                                                                                  
 tf.math.not_equal (TFOpLambda)  (None, 100, 1)      0           ['mask[0][0]']                   
                                                                                                  
 tf.cast (TFOpLambda)           (None, 100, 1)       0           ['tf.math.not_equal[0][0]']      
                                                                                                  
 tf.math.equal (TFOpLambda)     (None, 100, 1)       0           ['tf.cast[0][0]']                
                                                                                        

In [17]:
# Prepare model model saving directory.
import os
save_dir = 'model_checkpoints'
model_name = '%s_model.{epoch:03d}.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = keras.callbacks.ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = keras.callbacks.LearningRateScheduler(lr_schedule)
progress_bar = keras.callbacks.ProgbarLogger()
callbacks = [checkpoint, lr_scheduler, progress_bar]

In [18]:
train_dataset.shuffle()
model.fit(train_dataset.X, train_dataset.y,
          batch_size=batch_size,
#           epochs=epochs,
          epochs=1, # --- train only for 1 epoch here for demonstration ---
          validation_data=(val_dataset.X, val_dataset.y),
          shuffle=True,
          callbacks=callbacks)

[2023-04-07 06:07:41,697] INFO: Learning rate: 0.001000
2023-04-07 06:07:51.937760: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-04-07 06:07:54.156417: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f94d387feb0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-04-07 06:07:54.156490: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
2023-04-07 06:07:54.156507: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (1): Tesla P100-PCIE-16GB, Compute Capability 6.0
2023-04-07 06:07:54.156520: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (2): Tesla P100-PCIE-16GB, Compute Capability 6.0
2023-04-07 06:07:54.168849: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to 

      0/Unknown - 391s 0s/sample - loss: 0.2170 - accuracy: 0.9127WARNING:tensorflow:Can save best model only with val_acc available, skipping.


[2023-04-07 06:16:38,609] WARNING: Can save best model only with val_acc available, skipping.


1183/1183 [==============================] - 537s 454ms/sample - loss: 0.2170 - accuracy: 0.9127 - val_loss: 0.1898 - val_accuracy: 0.9240 - lr: 0.0010
